# Decoding Market Signals: Candlestick patterns for the S&P 500 index

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
from pathlib import Path
from collections import namedtuple
from scipy import stats

from BSquant import load_data
from BSquant import process_data
from BSquant import cs_pattern_recognition
from BSquant import cs_performance
from BSquant import plot_cs_performance
from BSquant import compute_trading_strategy_performance

pd.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2

# Loading data for the S&P500

In [ ]:
# Define the path to your ticker file
ticker_file = "./../data/SP500_tickers_one_per_line.txt"
notebooks_dir = Path("./../notebooks")
ticker_file_path = notebooks_dir.parent / "data" / ticker_file

tickers = []

# Open the ticker-file with a context manager and read each line adding ot to the list of tickers
with open(ticker_file_path, "r") as file:
    for line in file:
        ticker = line.strip()  # Strip newline characters and whitespace
        tickers.append(ticker)  # Add the cleaned ticker to the list

print("Number of tickers (may include multiple tickers per stock) is", len(tickers))
print("Number of unique tickers is:", set(tickers).__len__())

In [ ]:
# enumerated tickers in the ticker_file
for i, ticker in enumerate(tickers):
    print(f"{i+1}:{ticker}")

## Remove rows with missing data

In [ ]:
data_filename = "SP500_daily_data_1980_to_2023.csv.gz"
notebooks_dir = Path("./../notebooks")
data_file_path = notebooks_dir.parent / "data" / data_filename
print(data_file_path)

In [ ]:
%time
# passing a function as argument to another function; load all data we got
df = process_data(load_data(file_path=data_file_path, compression="gzip"))

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df["ticker"].unique().__len__()

We have data fo 501 stocks though in our ticker file we got 503 tickers. We obtained the tickers from a public source and their ticker formatting might be different from `CRSP`. This is not significant for our constex, however, it is worthwhile making sure the metadata of tradable securities is updated regularly. 

## Let us now compute for how many days per stock we got data for. 

Companies and be included and taken off the S&P 500 index. Some startups that were not previously listed might prosper and develop into companies large enough to be included in the index, while other may be outcompeted by others, cease to exist, acquired, split up, or taken private and hence either be excluded and/or delisted. Let us investigate for how many days we got data for each stock. While doing so, we find an interesting detour related to Python performance:

Python is primarily considered an interpreted language. Python code is executed by an interpreter, which reads the code at runtime and executes it line by line. This process is different from compiled languages, where the source code is transformed into machine code or bytecode before execution, typically resulting in an executable file. However, at a more detailed level, Python code is indeed compiled under the hood. More precisely, when Python code is executed, it is compiled into bytecode, which is a lower-level, platform-independent representation of the source code. This bytecode is then interpreted by the `Python Virtual Machine (PVM)`, however, compared to a purely compiled language such as `C` or `C++`, not turned into a standalone executable file. This process is automatic and transparent to the user, making Python feel like a purely interpreted language. Tools and third-party packages do exist that can package Python programs along with an interpreter into a standalone executable, but this is an additional step beyond Python's standard behavior.

The important point is that parsing byte-code through the PVM imposes an overhead which costs time. Hence, `Python` is considered "slow". However, you can use `C` and `C++` code within `Python` to leverage performance benefits. This is a common practice for computational heavy tasks where the execution speed of `Python` is a bottleneck. Integrating `C` or `C++` code into `Python` can significantly improve the performance of certain operations, especially those that are CPU-bound, such as numerical computations, data processing, and more.
This, however, required more detailed knowlege of the `Python compiler`, is not straight forward, and a topic for another repository. 

However, that does not mean we cannot speed up our code. Paricularly, we can make use of libraries that are written, at least partially, in `C` and available in Python, such as `numpy`. As `pandas` makes use of `numpy`, it is often possible to enjoy better performance, especially when we compute in-momory like we using `pandas`. Thus, it is generally good advise for the sake of performance, to "write highl-level code thinking low-level", and the following is meant to demonstrate this.

To compute the number of days we got for each of the S&P500 members, a streight-forward (but slow) method is to loop trough each ticker, filter the data frame according to the ticker, and to compute the number of rows. This will be executed below.

In [ ]:
%%time 

days_per_ticker = {}

for ticker in tickers:
    days_per_ticker[ticker] = df.query("ticker == @ticker").shape[
        0
    ]  # takes about 16.8 seconds
#     days_per_ticker[ticker] = df[df['ticker'] == ticker].shape[0]  # takes about 59.6 seconds and is three times slower, still.

As you see this step took approximately 16.8 seconds on the machine this code was executed on. Making use of the `pandas` native `.goupby()` method, which is written in `C`, and storing the results in a dictionary, achieves the same task in just about 112 ms, i.e, the computation is 150 times, i.e. an order of magnitude 
faster.  

In [ ]:
%%time 
days_per_ticker = df.groupby("ticker").size().to_dict();

## We now investigate how the length of the history of each stock [in days] is distributed

In [ ]:
plt.figure(figsize=(7, 7))
plt.hist(list(days_per_ticker.values()), bins=30)
plt.show()

In [ ]:
# Counter objects are a part of the collections module in Python's standard library.
# They are specialized dictionary subclasses designed to count hashable objects.
# A Counter is a collection where elements are stored as dictionary keys and their counts are stored
# as dictionary values.

Counter(list(days_per_ticker.values())).most_common(3)[
    0
]  # Counter(list(days_per_ticker.values())).most_common(3)[0][0] then extracts the number that occurs most often.

# 81 stocks contain 7881 days of data

## How are the stocks weighted with repect to the one with the longst history in the portfolio?

In [ ]:
max_days_ticker = max(
    days_per_ticker, key=days_per_ticker.get
)  # find the ticker with the maximum number of days
max_days = days_per_ticker[
    max_days_ticker
]  # retrieve the value (number of days) for this ticker
print(
    f"The ticker with the maximum number of days is: {max_days_ticker}, with {max_days} days."
)

max_days = max(days_per_ticker.values())  # find the maximum number of days
weights_per_ticker = {
    ticker: days / max_days for ticker, days in days_per_ticker.items()
}  # Calculate the weight for each ticker, z-transform the weights should you wish to use them in ML applications
weights_per_ticker

## Alternatively, we could define a start and end date ourselves and make sure to select only those stocks with a densely populated history.

Densely here means that the stocks should have the same number of data. This ensures stocks thate were recently taken in are not selected as they do not contain enough data

In [ ]:
selected_start_date = pd.Timestamp(2012, 1, 1)
selected_end_date = pd.Timestamp(2022, 12, 31)
df_filtered = df[
    (df["date"] >= selected_start_date) & (df["date"] <= selected_end_date)
]
df_filtered.groupby(
    "ticker"
).size().value_counts()  # 374 out of the 500 stocks are of the desired duration

If we wanted to just go for the mode directly, we could have achieved this by

In [ ]:
mode_size = df_filtered.groupby("ticker").size().mode()[0]
mode_size

And counted the number of stocks of that lenth using

In [ ]:
df_common_size = (
    df_filtered.groupby("ticker")
    .filter(lambda x: len(x) == mode_size)
    .reset_index(drop=True)
)

df_common_size["ticker"].unique().__len__()

If we selected only those stocks that have an equal amount of days between our start and end day, we have to reduce our universe from 500 stocks to 374. This is a significant reduction that one should be sure to afford.

As an alternative way, we appreaciate the different length of the data and conduct the pattern analysis for each of them separately.

# How does each stock evolve in time?

We limit ourselves to onem year of data to see how each of the stocks in the portfolio performed relative to their starting price

In [ ]:
def add_normalized_price(df: pd.DataFrame) -> pd.DataFrame:
    df["first_price_indicator"] = np.where(df.index == 0, 1, 0)
    df["first_price_value"] = df["first_price_indicator"] * df["close"]
    df["first_price_value"].replace(to_replace=0, method="ffill", inplace=True)
    df["normalized_price"] = df["close"] / df["first_price_value"]
    df.drop(columns=["first_price_indicator", "first_price_value"], inplace=True)
    return df

In [ ]:
# implementation using a multi-indexed data frame
result = df_common_size.set_index(["ticker", "date"]).join(
    df_common_size.groupby("ticker").first().add_prefix("first_")
)  # dg.set_index(['Date','ListingId']) will be equivalent to the vectorized version
result["normalized_price"] = result["close"] / result["first_close"]

In [ ]:
# plotting the data. Note you can select a nunber of stocks via the variable "counter" as well.

selected_start_date = pd.Timestamp(2022, 1, 1)
selected_end_date = pd.Timestamp(2022, 12, 31)

df_filtered = df[
    (df["date"] >= selected_start_date) & (df["date"] <= selected_end_date)
]
df_filtered

mode_size = df_filtered.groupby("ticker").size().mode()[0]

df_common_size = (
    df_filtered.groupby("ticker")
    .filter(lambda x: len(x) == mode_size)
    .reset_index(drop=True)
)

result = df_common_size.set_index(["ticker", "date"]).join(
    df_common_size.groupby("ticker").first().add_prefix("first_")
)  # dg.set_index(['Date','ListingId']) will be equivalent to the vectorized version
result["normalized_price"] = result["close"] / result["first_close"]

plt.figure(figsize=(10, 6))

counter = 0

for ticker, data in result.groupby(level="ticker"):
    plt.plot(
        data.index.get_level_values("date"), data["normalized_price"], label=ticker
    )
#     counter += 1
#     if counter == 20:
#         break

# plt.legend(title='Ticker', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Normalized price by date for the selected stocks")
plt.xlabel("date")
plt.ylabel("normalized price")
plt.tight_layout()
plt.show()

We see that the stock prices are not adapted to stock splits. As we work with intraday returns, we get away dealing with price-adjustments which, strictly speaking, is a topic in its own right. However, we do see the investment universe evolved, seemingly randomly, and there were constituents that performed positively, neutral, and negatively. 

Hence, selecting one stock in hindsight, and evaluating its buy-and-hold performance, is subject to bias. We will test in the following, how the candlestick patterns perform for the investment universe, to see whether they allow a a portfolion to be actively managed in a long-short fashion. 

# Candlestick analysis

We are now in a position to analyse the whole investment universe. Unfortunately, we now need to deal with a problem we so far go around with: "Big data analysis" comes with "big computational resources". 
Recall, we need to:
 - Loading data to memory coming from 500 stocks, with a history of up to 40 years,
 - Making the pattern recogniton logic act on them, where:
     - for every date, we create up to 61 new rows, where 61 is the number of candlestick patterns we are  able to identify using `talib`.
     
Unfortunately, this exceeds the memory resources of a standard workstation or laptop. 

In the following we hence limit ourselves to two years of data and outline how to proceed the analysis, but need to leave considering a wider time-interval to the interested reader whos has a more powerful machine available.

In [ ]:
# the path to the data file is the same as for the notbook discussing the single-stock case.
data_filename = "SP500_daily_data_1980_to_2023.csv.gz"
notebooks_dir = Path("./../notebooks")
data_file_path = notebooks_dir.parent / "data" / data_filename

df = process_data(
    load_data(
        data_file_path,
        selected_start_date=pd.Timestamp(2019, 1, 1),
        selected_end_date=pd.Timestamp(2022, 12, 31),
    )
)

In [ ]:
%%time
cs_signals_df = cs_pattern_recognition(df=df)

In [ ]:
%%time
performance_metrics = cs_performance(cs_signals_df)

In [ ]:
# plot all patterns, ranked by number of instances
plot_cs_performance(
    df=performance_metrics,
    criterion="total_instances",
    title_suffix="across the whole data set.",
)

# plot the patterns, ranked by number of instances, with a true-positive rate >50%.
plot_cs_performance(
    df=performance_metrics.query("ci_lower > 0.5").sort_values(
        by="total_instances", ascending=False
    ),
    criterion="total_instances",
    title_suffix="with ci_lower > 50%.",
)

Notable this time, appreciating more data, we indeed isntances where the lower part of the confidence interval is greater than 50%. This tells us we are 95% sure that these patterns indeed correctly predict the next day's intraday return. Conversely, we are now also in a position to identify counter-signals. These are instances where the upper part of the confidence interval is below the 50% threshold. Hence, it is indicated to take action for these signals in opposite direction of what they suggest, i.e. to take them as contrarian signals. 

Let us visualise these reults in the following.

In [ ]:
performance_metrics.query("ci_lower > 0.5").sort_values(
    by=["ci_lower"], ascending=False
)

In [ ]:
performance_metrics.query("ci_upper < 0.5").sort_values(
    by=["ci_upper"], ascending=False
)

In [ ]:
plot_cs_performance(
    df=performance_metrics, criterion="TP_wilson", plot_performance=True
)

Based on this analysis, we now name signals and contrarian signals:

In [ ]:
performance_metrics.query("ci_lower > 0.5").index  # signals

In [ ]:
performance_metrics.query("ci_upper < 0.5").index  # anti signals

If you want, you can test strategies that contain only those candlestick patterns that have proven to be profitable and/or those which manifsted themselves as anti-signals. 

Also you now can implement your own Machine-Learning logics to see whether you can come up with your own logic. Also, you can run the logic on a more potent machine, to see how the precision and confidence intervals change per candlestick pattern. 

# Applying candlestick analysis across the S&P 500 universe

Unfortunately, the data provider does not have OHLC data for the S&P 500 index. At the time of writing, an inquiry is still ongoing. In the following, we illustrate how we obtain a synthetic performance reference nontheless, namely by computing the mean intraday return at any day acroos all the oniverse assuming equal weights. This shoudl serve as an approximate solution that works with the data at hand.

In [ ]:
# synthetic S&P 500 intraday performacne
df_reference_strategy = (
    df[["ticker", "date", "intraday_return"]]
    .pivot_table(index="date", columns="ticker")
    .mean(axis=1)
)
df_reference_strategy = df_reference_strategy.rename("intraday_return").reset_index()
df_reference_strategy["account_curve"] = (
    1 + df_reference_strategy["intraday_return"]
).cumprod()
df_reference_strategy["cumsumret"] = df_reference_strategy["intraday_return"].cumsum()
df_reference_strategy.plot(x="date", y="account_curve", figsize=(8, 8))
plt.show()

compute_trading_strategy_performance(df=df_reference_strategy, verbose=True);

However, we can also opt for a method that compares the buy-and hold approach against employign the naive candlestick strategy. Recall the naive candlestick approach was to sum up the signal for each day across all the candlestick patterns. We can then compare the Sharpe Ration for both, the buy-and-hold approach, as well as the active candlestick approach and compare which of them delivers higher risk-adjusted returns, if at all.
This is carrie dout in the below, whearas we slightly modified the approach of the single-stock method from notebook 2. 

## Analysing Sharpe Ratios for a passive and acive trading strategy for the S&P 500 universe

We now turn our focus to comparing the Sharpe Ratios from both, passive and active trading strategies, examining their distribution through histograms, cumulative distribution functions, and box plots. Additionally, we employ specific functions for a detailed statistical analysis. 

Precisely, `analyse_Sharpe_Ratios_for_active_and_passive_strategies()` visualizes their distributions, `analyze_sharpe_ratios()` delves into their statistical characteristics, and `compare_sharpe_ratios()` statistically determines if the active strategy's Sharpe Ratios significantly outperform those of a reference strategy.

In [ ]:
def analyse_Sharpe_Ratios_for_active_and_passive_strategies(
    SR_buy_and_hold: np.array, SR_naive_cs: np.array
) -> None:
    # determine the combined range of both Sharpe Ratios
    all_ratios = np.concatenate((SR_buy_and_hold, SR_naive_cs))
    min_edge = all_ratios.min()
    max_edge = all_ratios.max()
    bins = np.linspace(
        min_edge, max_edge, 40
    )  # 40 equal-width bins across the full range

    # compute empirical CDF for SR_buy_and_hold
    sorted_SR_buy_and_hold = np.sort(SR_buy_and_hold)
    yvals_buy_and_hold = np.arange(1, len(sorted_SR_buy_and_hold) + 1) / float(
        len(sorted_SR_buy_and_hold)
    )

    # compute empirical CDF for SR_naive_cs
    sorted_SR_naive_cs = np.sort(SR_naive_cs)
    yvals_naive_cs = np.arange(1, len(sorted_SR_naive_cs) + 1) / float(
        len(sorted_SR_naive_cs)
    )

    # plotting
    fig, axs = plt.subplots(3, 1, figsize=(10, 18))

    # histograms
    axs[0].hist(SR_buy_and_hold, bins=bins, alpha=0.5, label="Buy and hold")
    axs[0].hist(SR_naive_cs, bins=bins, alpha=0.5, label="Naive CS")
    axs[0].set_title("Comparison of Sharpe Ratios: Buy and hold vs. Naive CS")
    axs[0].set_xlabel("Sharpe Ratio")
    axs[0].set_ylabel("Frequency")
    axs[0].legend()

    # empirical CDFs
    axs[1].plot(
        sorted_SR_buy_and_hold,
        yvals_buy_and_hold,
        label="Buy and hold",
        marker=".",
        linestyle="none",
    )
    axs[1].plot(
        sorted_SR_naive_cs,
        yvals_naive_cs,
        label="Naive CS",
        marker=".",
        linestyle="none",
    )
    axs[1].set_title("Empirical CDF of Sharpe Ratios")
    axs[1].set_xlabel("Sharpe Ratio")
    axs[1].set_ylabel("CDF")
    axs[1].legend()
    axs[1].grid(True)

    # box Plots
    axs[2].boxplot([SR_buy_and_hold, SR_naive_cs], labels=["Buy and hold", "Naive CS"])
    axs[2].set_title("Box Plot of Sharpe Ratios")
    axs[2].set_ylabel("Sharpe Ratio")

    plt.tight_layout()
    plt.show()

    return None


def analyze_sharpe_ratios(SR_buy_and_hold: np.array, SR_naive_cs: np.array) -> None:
    print("Buy and Hold Strategy:")
    print(f"Mean Sharpe Ratio: {np.mean(SR_buy_and_hold):.4f}")
    print(f"Median Sharpe Ratio: {np.median(SR_buy_and_hold):.4f}")
    print(f"Kurtosis: {stats.kurtosis(SR_buy_and_hold):.4f}")
    print(f"Skewness: {stats.skew(SR_buy_and_hold):.4f}\n")

    print("Naive Candlestick Strategy:")
    print(f"Mean Sharpe Ratio: {np.mean(SR_naive_cs):.4f}")
    print(f"Median Sharpe Ratio: {np.median(SR_naive_cs):.4f}")
    print(f"Kurtosis: {stats.kurtosis(SR_naive_cs):.4f}")
    print(f"Skewness: {stats.skew(SR_naive_cs):.4f}")

    return None


def compare_sharpe_ratios(SR_buy_and_hold: np.array, SR_naive_cs: np.array) -> None:
    t_stat, p_value = stats.ttest_ind(
        SR_naive_cs, SR_buy_and_hold, alternative="greater"
    )

    print(f"t-statistic: {t_stat}")
    print(f"p-value: {p_value}")

    # Interpret the p-value
    if p_value < 0.05:
        print(
            "The naive candlestick strategy has significantly greater Sharpe Ratios than the buy-and-hold strategy at the 5% significance level."
        )
    else:
        print(
            "There is no significant difference in Sharpe Ratios in favour of the naive candlestick strategy over the buy-and-hold strategy at the 5% significance level."
        )

        return None

## First approach: Taking into account all candlestick patterns

Now, we compute the Sharpe Ratios for the active, as well as passive (reference) strategy. 

In [ ]:
%%time 

# we should loop through all of the tickers to create trading signals for each stock

StrategyPerformance = namedtuple(
    "StrategyPerformance", ["SR_buy_and_hold", "SR_naive_cs"]
)
naive_cs_vs_buy_and_hold_performance = {}

for ticker in tickers:
    df_single_stock = df[df["ticker"] == ticker]
    cs_single_stock_signals_df = cs_signals_df[cs_signals_df["ticker"] == ticker]

    trading_signal = (
        cs_single_stock_signals_df.query("cs_pattern != 0")
        .pivot_table(index="date", columns="candle", values="cs_pattern", aggfunc="sum")
        .sum(axis=1)
        .loc[lambda x: x != 0]
    )

    performance_trading_signals = (
        df_single_stock[
            df_single_stock["date"].isin(
                [date + pd.DateOffset(days=1) for date in trading_signal.index]
            )
        ][["date", "intraday_return"]]
        .assign(account_curve=lambda x: (1 + x["intraday_return"]).cumprod())
        .assign(cumsumret=lambda x: x["intraday_return"].cumsum())
        .assign(time_between_signals=lambda x: x["date"].diff().dt.days)
    )

    (_, _, SR_buy_and_hold) = compute_trading_strategy_performance(df=df_single_stock)
    (_, _, SR_naive_cs) = compute_trading_strategy_performance(
        df=performance_trading_signals
    )

    naive_cs_vs_buy_and_hold_performance[ticker] = StrategyPerformance(
        SR_buy_and_hold=SR_buy_and_hold, SR_naive_cs=SR_naive_cs
    )

In [ ]:
SR_buy_and_hold = np.array(
    [
        performance.SR_buy_and_hold
        for performance in naive_cs_vs_buy_and_hold_performance.values()
        if not np.isnan(performance.SR_buy_and_hold)
    ]
)
SR_naive_cs = np.array(
    [
        performance.SR_naive_cs
        for performance in naive_cs_vs_buy_and_hold_performance.values()
        if not np.isnan(performance.SR_naive_cs)
    ]
)

analyse_Sharpe_Ratios_for_active_and_passive_strategies(
    SR_buy_and_hold=SR_buy_and_hold, SR_naive_cs=SR_naive_cs
)

In [ ]:
analyze_sharpe_ratios(SR_buy_and_hold=SR_buy_and_hold, SR_naive_cs=SR_naive_cs)

Notably, both the mean and the median Sharpe Ratio for the naive candlestick approach (active trading strategy) are higher than for the passive (buy-and-hold) approach. Interestingly, also the skewness is slighly lower for the active approach. Recall that skewness values close to zero suggest a symmetrical distribution of returns around the mean. For the buy-and-hold strategy, a skewness of -0.3861 indicates a skew to the left, suggesting a distribution with a fatter left tail indicating more frequent negative extreme returns than positive ones. The candlestick approach's skewness of -0.2999 also indicates a leftward skew but to a lesser extent, suggesting a slightly more symmetric distribution of returns around the mean compared to the buy-and-hold strategy. 

Unfortunately, the extra returns obtained form the candlestick approach do not come for free: They are also more risk as indicated by the larger kurtosis. Also note we did not assume any transation costs. For a trading company, a market maker, or a large bank, which are in a position to negotiate lower transaction costs than retail traders, these are less of an issue. However, an active approach involving daily transactions for a retail trader is disadvantageous, to say the least. The active candlestick strategy hence appears more suitable for risk seeking speculatnts who are in pursuit of "high-risk-high-return" bets. 

However, it is noted that the outperformace of mean return, median return, and a smaller skew for the active candlestick approach are indeed interesting observations. In order to determine whether higher Sharpe Ratios for the candlestick approach are statistically significantly greater than the ones for the passive strategy, we perform a one-sided t-test.

In [ ]:
compare_sharpe_ratios(SR_buy_and_hold, SR_naive_cs)

## Second approach: Filter only statisticaly significant cs patterns

In [ ]:
positive_signals = performance_metrics.query("ci_lower > 0.5").index  # signals
counter_signals = performance_metrics.query(
    "ci_upper < 0.5"
).index  # anti-signals/contrarians

In [ ]:
%%time 

StrategyPerformance = namedtuple(
    "StrategyPerformance", ["SR_buy_and_hold", "SR_naive_cs"]
)
naive_cs_vs_buy_and_hold_performance = {}

for ticker in tickers:
    df_single_stock = df[df["ticker"] == ticker]
    cs_single_stock_signals_df = cs_signals_df[cs_signals_df["ticker"] == ticker]

    # create a copy for modification
    filtered_signals_df = cs_single_stock_signals_df.copy()

    # apply the filter directly to this copy
    filter_mask = filtered_signals_df.index.get_level_values("candle").isin(
        positive_signals.union(counter_signals)
    )
    filtered_signals_df = filtered_signals_df.loc[filter_mask]

    # adjust 'cs_pattern' by multiplying by -1 for counter signals
    counter_signals_mask = filtered_signals_df.index.get_level_values("candle").isin(
        counter_signals
    )
    filtered_signals_df.loc[counter_signals_mask, "cs_pattern"] *= -1

    trading_signal = (
        filtered_signals_df.query("cs_pattern != 0")
        .pivot_table(index="date", columns="candle", values="cs_pattern", aggfunc="sum")
        .sum(axis=1)
        .loc[lambda x: x != 0]
    )

    performance_trading_signals = (
        df_single_stock[
            df_single_stock["date"].isin(
                [date + pd.DateOffset(days=1) for date in trading_signal.index]
            )
        ][["date", "intraday_return"]]
        .assign(account_curve=lambda x: (1 + x["intraday_return"]).cumprod())
        .assign(cumsumret=lambda x: x["intraday_return"].cumsum())
        .assign(time_between_signals=lambda x: x["date"].diff().dt.days)
    )

    (_, _, SR_buy_and_hold) = compute_trading_strategy_performance(df=df_single_stock)
    (_, _, SR_naive_cs) = compute_trading_strategy_performance(
        df=performance_trading_signals
    )

    naive_cs_vs_buy_and_hold_performance[ticker] = StrategyPerformance(
        SR_buy_and_hold=SR_buy_and_hold, SR_naive_cs=SR_naive_cs
    )

In [ ]:
SR_buy_and_hold = np.array(
    [
        performance.SR_buy_and_hold
        for performance in naive_cs_vs_buy_and_hold_performance.values()
        if not np.isnan(performance.SR_buy_and_hold)
    ]
)
SR_naive_cs = np.array(
    [
        performance.SR_naive_cs
        for performance in naive_cs_vs_buy_and_hold_performance.values()
        if not np.isnan(performance.SR_naive_cs)
    ]
)

In [ ]:
analyze_sharpe_ratios(SR_buy_and_hold=SR_buy_and_hold, SR_naive_cs=SR_naive_cs)

In [ ]:
compare_sharpe_ratios(SR_buy_and_hold, SR_naive_cs)

Recall from above, the naive Candlestick strategy utilizing all patterns was characterised by the following performacne metrics:

```
Mean Sharpe Ratio: 0.2325
Median Sharpe Ratio: 0.2617
Kurtosis: 1.7315
Skewness: -0.2999
```

# Conclusion

Upon filtering the candlestick signals and contrarian signals that were found to be indicating a price move on the next day at the 5% significance level, we could improve the all performance statistics, based on the Sharpe Ratio, in all four categories examined. The full set consists of 61 candlestick patterns, whereas the filtered approach consists of 12 signals and 15 counter signals. 

The signals were identified at the 5% significance level to be:

```
Index(['CDLDOJI', 'CDLLONGLEGGEDDOJI', 'CDLRICKSHAWMAN', 'CDLHAMMER',
       'CDLMATCHINGLOW', 'CDLINVERTEDHAMMER', 'CDLGRAVESTONEDOJI',
       'CDLSEPARATINGLINES', 'CDLSTICKSANDWICH', 'CDLUNIQUE3RIVER',
       'CDLCOUNTERATTACK', 'CDL3LINESTRIKE'],
      dtype='object', name='candle')
```

The contrarian signals were identified at the 5% significance level to be:
```
Index(['CDLSPINNINGTOP', 'CDLLONGLINE', 'CDLBELTHOLD', 'CDLCLOSINGMARUBOZU',
       'CDLSHORTLINE', 'CDLENGULFING', 'CDLMARUBOZU', 'CDL3OUTSIDE',
       'CDLSHOOTINGSTAR', 'CDLEVENINGSTAR', 'CDLDARKCLOUDCOVER', 'CDLPIERCING',
       'CDLEVENINGDOJISTAR', 'CDLTRISTAR', 'CDL2CROWS'],
      dtype='object', name='candle')
```

Notably, the filtered candlestick approach outperforms the naive buy strategy in all four performance categories investigated. Moreover, a one-sided `t-test` revealed that the Sharpe Ratios obtained by the filtered candlestick approach is greated than those obtained by the naive buy approach at the 5% level.

In further research, one could run the very same code on a more potent machine and simply select a longer data range when loading in the data, to see whether the results reported here still hold. Also, one could attempt an expanding-window approach regarding the considered time-frame to investigate how performance changes in time and whether there are stocks for which the candlestick approach works particularly good or bad. The data source considered for this analysis were the past two years for all S&P 500 components, although for some stocks, there exists data dating back to the 1980ies. Assuming a densely populated data this equates to an upper boundary of 20 000 years of daily stock OHLC data.


It remains, however, that the level of analysis carried out and presented here, required access to proprietary data and significant computing power. These could be, for example, the High Performance Computing (HPC) facilities at Imperial College, or a potent private institution. Moreover, an active trading approach is predominantly aimed at players like large hedge funds and investment banks which still have proprietary trading teams, and that are in a position to negotiate low transaction costs. They should also be potent enough to observe, and act, on data streams across the entire S&P 500 universe. An extention to any other index, such as the STOXX600 or any Asian index is easily diable using the existing code. 

For fund managers, with a more passive approach, the presented analysis can be interesting to optimise their entry points opon which to accummulate or offload positions. 

For brokers, the presented analysis is useful to craft an arrival strategies to which to adopt their execution logics based on an oppinion whether a stock goes up or down. In case of no signal on a particularly stock, one then would simply fall back to a default behaviour.   

# END

# Appendix

## ML approach

The author cannot run the ML approach as we do not have enough memory available to load the required history of the stocks. ML methods are inherently data hungy, so loading just two years of data per stock will not be enough for meaningful results. Also, we cannot mix the hostory of one stock with the history of another, as financial data is chronological in nature. 